In [1]:
import numpy as np
import cv2
import glob
import os

In [147]:
chess_path = "chess"

In [148]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((7*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:7].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob(os.path.join(chess_path, '*.jpg'))
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    # here I used findChessboardCornersSBWithMeta function, it performs better with my images
    ret, corners, meta = cv2.findChessboardCornersSBWithMeta(gray, (7,7), flags=cv2.CALIB_CB_LARGER+ cv2.CALIB_CB_NORMALIZE_IMAGE)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        #increaqse accuracy of found corners
        corners2=cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv2.drawChessboardCorners(img, (7,7), corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(0)
cv2.destroyAllWindows()

### Calibration

In [149]:
#return camera matrix, distortion parameters, rotation and translation vectors
ret, cameraMatrix, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print("Camera Calibrated: ", ret)
print("\nCamera Matrix:\n", cameraMatrix)
print("\nDistortion Parameters:\n", dist)
print("\nRotation Vectors:\n", rvecs)
print("\nTranslation Vectors:\n", tvecs)

Camera Calibrated:  2.526804284145464

Camera Matrix:
 [[280.89121204   0.         320.72547099]
 [  0.         267.29195368 213.2422601 ]
 [  0.           0.           1.        ]]

Distortion Parameters:
 [[-0.1992655   0.10928112  0.02518497 -0.0067436  -0.0350769 ]]

Rotation Vectors:
 (array([[-0.14401081],
       [-0.05004611],
       [-0.0090575 ]]), array([[-0.14286949],
       [ 0.15585857],
       [-0.36099374]]), array([[-0.13971267],
       [ 0.21817619],
       [ 0.04018194]]), array([[ 0.00479278],
       [-0.26894791],
       [-0.67521536]]), array([[-0.20696361],
       [ 0.24458317],
       [ 0.4167562 ]]), array([[-0.76307894],
       [ 0.19723718],
       [ 0.58027485]]), array([[-0.56155612],
       [-0.07038868],
       [ 0.0524318 ]]), array([[-0.04428237],
       [ 0.2828991 ],
       [-0.00638895]]), array([[-0.6130443 ],
       [-0.31266662],
       [-0.65226509]]), array([[-0.56960749],
       [-0.15899887],
       [-0.4711167 ]]), array([[-0.07621002],
      

### Undistortion

I have changed **findChessboardCorners** function to **cv2.findChessboardCornersSBWithMeta** and my corner detector started to find corners in all images. After undistortion image looks better in general (just table looks strange).

In [150]:
#take new image
img = cv2.imread('chess/chess7.jpg')
h,  w = img.shape[:2]
newCameraMatrix, roi = cv2.getOptimalNewCameraMatrix(cameraMatrix, dist, (w,h), 1, (w,h))

In [151]:
# undistort
dst = cv2.undistort(img, cameraMatrix, dist, None, newCameraMatrix)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibrated/calibresult.png', dst)

True

### Re-Projection Error

In [152]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], cameraMatrix, dist)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )

total error: 0.25977566493690624


### 3D rendering using calibrated camera

Function which takes the corners in the chessboard (obtained using cv.findChessboardCorners()) and axis points to draw a 3D axis.

In [153]:
def draw(img, corners, imgpts):
    #convert coordinates to int
    corners = np.int32(corners)
    #take first corner
    corner = tuple(corners[0].ravel())
    imgpts = np.int32(imgpts)

    img = cv2.line(img, corner, tuple(imgpts[0].ravel()), (255,0,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel()), (0,0,255), 5)
    return img

In [154]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objp = np.zeros((7*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:7].T.reshape(-1,2)
axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)

In [155]:
for fname in glob.glob(os.path.join(chess_path, '*.jpg')):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners, meta = cv2.findChessboardCornersSBWithMeta(gray, (7,7), flags=cv2.CALIB_CB_LARGER+ cv2.CALIB_CB_NORMALIZE_IMAGE)
    if ret == True:
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        # Find the rotation and translation vectors.
        ret, rvecs, tvecs = cv2.solvePnP(objp, corners2, cameraMatrix, dist)
        # project 3D points to image plane
        imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, cameraMatrix, dist)
        # draw axes from the first found cormer to projected points in 3D
        img = draw(img, corners2, imgpts)
        cv2.imshow('img',img)
        k = cv2.waitKey(0)
        fname = fname.split("\\")[1].split(".")[0]
        cv2.imwrite("calibrated/"+fname+'_3d.png', img)
cv2.destroyAllWindows()